In [1]:
import numpy as np
import pandas as pd

In [2]:
# Table 2 
temp_df = pd.read_excel('/n/holystore01/LABS/price_lab/Users/mjzhang/scTRS_data/single_cell_data/'
                        'mouse_liver_halpern_nature_2017/SuppTable3_zonation.xlsx', skiprows=2, index_col=0)


In [3]:
df_hom = pd.read_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scTRS_data/gene_annotation/'
                     'mouse_human_homologs.txt', sep='\t')
dic_map = {x:y for x,y in zip(df_hom['MOUSE_GENE_SYM'], df_hom['HUMAN_GENE_SYM'])}

def convert_gene(x, dic_map):
    for gene in str(x).split(';'):
        if gene in dic_map:
            return dic_map[gene]
    return 'nomap'

In [4]:
df_zon = temp_df.loc[temp_df['q-values']<0.01]
df_zon.index = [convert_gene(x, dic_map) for x in df_zon.index]
df_zon = df_zon[['Layer %d'%x for x in range(1,10)]]
df_zon['Corr'] = [np.corrcoef(df_zon.loc[x], np.arange(9))[0,1] for x in df_zon.index]
df_zon['MeanExpr'] = df_zon[['Layer %d'%x for x in range(1,10)]].max(axis=1)
df_zon = df_zon.loc[(df_zon['MeanExpr']>0.0001) & (np.absolute(df_zon['Corr'])>0.6)]
df_zon.sort_values('Corr', inplace=True)

dic_gs = {}
dic_gs['CV'] = list(df_zon.index[df_zon['Corr']<0])
dic_gs['PN'] = list(df_zon.index[df_zon['Corr']>0])

In [5]:
df_gs = pd.DataFrame(columns=['TRAIT', 'GENESET'])
for term in dic_gs:
    df_gs.loc[term] = [term, ','.join(dic_gs[term])]
df_gs.to_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scTRS_data/gs_file/zonation_halpern_2017.gs',
             sep='\t', index=False)

In [6]:
df_gs

,TRAIT,GENESET
CV,CV,"RPS18,SMARCA2,ACOT2,KLF13,PPARA,FERMT2,PSMD4,C..."
PN,PN,"SCD,ST3GAL5,TAT,SDHC,ARG1,LEAP2,CCDC142,CXCL12..."
